In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

#### Loading Dataset

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [5]:
X_train.shape, y_train.shape

((60000, 28, 28), (60000,))

In [6]:
X_train = X_train.reshape(-1, 28*28).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28*28).astype('float32') / 255.0

#### Sequential API
(Very convient, not very flexible)

In [7]:
model = keras.Sequential(
    [
        layers.Input(shape=(28*28,)),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(10)
    ]
)

In [12]:
# model = keras.Sequential()
# model.add(layers.Input((28*28,)))
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dense(10))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 10)                2570      
                                                                 
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

In [10]:
model.fit(X_train, y_train, batch_size=32, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 12s 5ms/step - loss: 0.1851 - accuracy: 0.9442
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0790 - accuracy: 0.9752
Epoch 3/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0534 - accuracy: 0.9827
Epoch 4/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0409 - accuracy: 0.9866
Epoch 5/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0332 - accuracy: 0.9895


In [11]:
model.evaluate(X_test, y_test, batch_size=32, verbose=2)

313/313 - 2s - loss: 0.0779 - accuracy: 0.9788 - 2s/epoch - 5ms/step


[0.07790038734674454, 0.9787999987602234]

#### Functional API

In [16]:
inputs = layers.Input(shape=(28*28,))
x = layers.Dense(512, activation='relu', name='first_layer')(inputs)
x = layers.Dense(256, activation='relu', name='second_layer')(x)
outputs = layers.Dense(10)(x)

model = keras.Model(inputs, outputs)

In [17]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 784)]             0         
                                                                 
 first_layer (Dense)         (None, 512)               401920    
                                                                 
 second_layer (Dense)        (None, 256)               131328    
                                                                 
 dense_4 (Dense)             (None, 10)                2570      
                                                                 
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

r = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=5,
    validation_data = (X_test, y_test)
)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1893 - accuracy: 0.9426 - val_loss: 0.0908 - val_accuracy: 0.9709
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0805 - accuracy: 0.9752 - val_loss: 0.1049 - val_accuracy: 0.9658
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0560 - accuracy: 0.9820 - val_loss: 0.0874 - val_accuracy: 0.9745
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0422 - accuracy: 0.9868 - val_loss: 0.0831 - val_accuracy: 0.9761
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0323 - accuracy: 0.9900 - val_loss: 0.0874 - val_accuracy: 0.9770


#### Extracting Specific layer features

In [20]:
model = keras.Model(inputs=model.inputs,
                    outputs=[model.layers[-2].output]) # ---> by indexing

# [model.get_layers('layer_name').output] ---> get layers through layer name
# [layer.output for layer in model.layers] ---> for all layers

In [21]:
feature = model.predict(X_train)

1875/1875 [==============================] - 2s 1ms/step


In [23]:
feature.shape

(60000, 256)